# Init

In [ ]:
# To work with files
import os

# To use DataFrames
import pandas as pd

# Used in funct def f_comp_angle(row):
import math

# To plot graphs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib nbagg

# To allow more outputs in Jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Path to Actigraphy data
path = "C:\\Users\\sigmu\\Desktop\\dataset\\"
path_ACG = path + "vlastni\\Aktigrafie po dnech\\"
files_ACG = os.listdir(path_ACG)

# good accuracy parameters: first_threshold = 10 minutes, time_window = 3 minutes, angle = 10 (divided by 5 ... 5 s resampling)
first_threshold = (10 * 60) / 5
time_window = (3 * 60) / 5
angle = 10

# dataframe to write the result to
df_stats = pd.DataFrame(columns=['Participant',
                                 'TIB', 'SOL', 'TST', 'WASO', 'NA>5', 'SFI', 'SWR', 'SE%'])

# Functions

In [ ]:
# Function to compute angle according to van Hees method

def f_comp_angle(row):
    return math.degrees(math.atan(row['z axis [g]'] / ((row['x axis [g]']**2 + row['y axis [g]']**2)**0.5)))

# -----------------------------------------------------------------------------------------------------------  

# Function to decide sleep & wake epochs, first sleep time and sleep end time, if the accelerometer has been worn, 
# sleep fragmentation and number of awakenings > 5
# parameters: window of inactivity to decide SO, window to decide sleep after SO (both in seconds / 5 ... 5s resampling), 
# angle threshold and ACG dataframe

def f_inactiv(first_threshold, time_window, angle, df):
    sleep_onset = sleep_end = sleep_fragmentation = counter = non_wear_counter = five_min = NA_5 = 0
    first_sleep = woke_up = frag = False
    result = []
    for index, value in df['abs_angle_change'].items():
        counter += 1

        # Non-wear possible solution: if angle change is 0 for at least an hour
        if (value < 0.1):
            non_wear_counter += 1
            if(non_wear_counter == 720):
                df['ACG_state'] = "N"      
                break
        else:
            non_wear_counter = 0
                
        # Angle change > angle -> woke up
        if (value > angle):
            counter = 0                                 
        # After first sleep  
        elif (counter > time_window) & first_sleep:
            # Write S to state
            df.loc[index, 'ACG_state'] = "S"            
            five_min = 0
            frag = False
            woke_up = True
            # Sleep End (end of sleep period)
            sleep_end = index
        # First sleep 
        elif (counter > first_threshold) & ~first_sleep:
            # Write S to state
            df.loc[index, 'ACG_state'] = "S"
            first_sleep = True
            # Sleep Onset (start of sleep period)
            sleep_onset = index  
            
        # after SO: if woken up, add to Sleep Fragmentation count
        if ((df.loc[index, 'ACG_state'] == 'W') & first_sleep):                                          
            if(frag == False):
                sleep_fragmentation += 1
                frag = True  
            
            five_min += 1
            # if greater than 5 minutes, add to NA>5
            if ((five_min > (5*60/5)) & woke_up):
                NA_5 += 1  
                woke_up = False
            
    result.append(sleep_onset)
    result.append(sleep_end)
    result.append(sleep_fragmentation)
    result.append(NA_5)
    return result
 
# -----------------------------------------------------------------------------------------------------------

# Function to compute sleep parameters
# parameters: ACG dataframe, result dataframe to write statistics in
def f_stats(df, result):    
    
    # If non-wear
    if(df['ACG_state'][0] == 'N'):
        print("Accelerometr is not being used in recording: " + chosen_participant)
        return
    
    # TIB is presumed to be whole recording after f_cut_start, f_cut_end, in minutes
    TIB = (df.index[len(df)-1] - df.index[0]) / np.timedelta64(1,'m')
    
    # Sleep Onset and last Sleep from f_inactiv func (as timestamp -> str)
    try:
        sleep_onset = str(inactivity[0].time())    
        sleep_end = str(inactivity[1].time())
        last = df['ACG_state'][len(df)-1]
        sleep_end_greater = df.index[len(df)-1] < inactivity[1]
        # If last item in dataframe is sleep and it is less than sleep_end (sleep_end was created with 5s epochs - could be after last item)
        if ((last == 'S') & sleep_end_greater):
            sleep_end = str(df.index[len(df)-1].time())
        # Sleep Onset Latency: sleep_onset - start of recording
        SOL = inactivity[0] - df.index[0]
        SOL = round(SOL.seconds / 60, 2)
    except:
        sleep_onset = 'NaN'
        sleep_end = 'NaN'
        SOL = 'NaN'
        print("SOL is not registered in recording: " + chosen_participant)
    
    # TST is the duration in minutes of all sleep epochs between SOL time and sleep end
    TST = (len(df[df['ACG_state']=='S'].index.value_counts()) * 30 ) / 60
    
    # WASO is the duration in minutes of all wake epochs between SOL time and sleep end
    WASO = ( len(df[(df['ACG_state']=='W')&(df.index > inactivity[0])]) * 30) / 60
    
    # NA>5 is number of awakenings after SO with duration greater than 5 minutes
    try:
        NA_5 = inactivity[3]
    except:
        NA_5 = 'NaN'
    
    # Sleep Wake Ratio
    try:
        SWR = round(TST / WASO, 2)
    except:
        SWR = 'NaN'       
    
    # Sleep Eficiency
    try:
        SE = round((TST / TIB)*100, 2)
    except:
        SE = 'NaN'
    
    # Sleep Fragmentation is number of intervals scored as "W" (after sleep onset) relative to the total sleep time in hours
    try:
        SFI = round(inactivity[2] / (TST/60), 2)
    except:
        SFI = 'NaN'    
    
    # Add to stats dataframe
    result = result.append({'Participant': participant,
                            'TIB': TIB, 'SOL': SOL, 'TST': TST, 'WASO': WASO, 'NA>5': NA_5, 'SFI': SFI, 'SWR': SWR, 'SE%': SE},
                           ignore_index=True)
    
    return result

# Iterate through files

In [ ]:
%%time

# go through files
for file_name_ACG in files_ACG:
    # split on _
    participant = file_name_ACG.split("_")[0]
    print(participant)

    df = pd.read_csv(path_ACG+file_name_ACG, names=['time stamp', 'x axis [g]', 'y axis [g]', 'z axis [g]', 
                                                    'light level [lux]', 'button [1/0]','temperature [°C]'],
                     skiprows=100,
                     # might be slightly faster:
                    infer_datetime_format=True, memory_map=True)
    df['time stamp'] = pd.to_datetime(df['time stamp'], format='%Y-%m-%d %H:%M:%S:%f')

    # drop not used columns from ACG
    df.drop(columns=['light level [lux]', 'button [1/0]', 'temperature [°C]'], inplace=True, axis=1)
    
    #-----------------------------------------------------------------------------------------------  
    
    # Resample by 5 second epoch and compute median of x,y,z
    df = df.resample('5S', on='time stamp', kind='timestamp').median()#.round(decimals=4)

    #-----------------------------------------------------------------------------------------------  

    # Apply func comp_angle
    df['angle'] = df.apply(f_comp_angle, axis=1)#.round(decimals=4)

    # Return absolute difference in angle per column
    df['abs_angle_change'] = df['angle'].diff().abs()#.round(decimals=4)

    # New column with all "W" values
    df['ACG_state'] = "W"

    #-----------------------------------------------------------------------------------------------    

    # Decide inactivity based on the angle 
    # returns list of timestamps for SE, SO
    inactivity = f_inactiv(first_threshold, time_window, angle, df)

    if(df['ACG_state'][0] != 'N'):

        #-----------------------------------------------------------------------------------------------

        # Resample ACG to have same number of columns as PSG (30s epochs)
        df = df.resample('30S').interpolate()
        
        #-----------------------------------------------------------------------------------------------

        # Compute statistics
        df_stats = f_stats(df, df_stats)

    else:
        print("Accelerometr is not being used.")

# Write to file

In [ ]:
returns_path_csv = path + "Results\\" + "vlastni_final_STATS.csv"
df_stats.to_csv(returns_path_csv)

# Read from file

In [ ]:
read_path_csv = path + "Results\\"+ "vlastni_final_STATS.csv"
stats = pd.read_csv(read_path_csv)
stats

In [ ]:
# Correlation Matrix
## Understand the dependence between the independent variables of the data set.
## Helps choose important and non-redundant variables of the data set.
## Applicable only to numeric/continuous variables.
import seaborn as sn
 
# Numeric columns of the dataset
numeric_col = ['TIB', 'SOL', 'TST', 'WASO', 'NA>5', 'SWR', 'SFI', 'SE%']
 
# Correlation Matrix formation
corr_matrix = stats.loc[:,numeric_col].corr()
print(corr_matrix)
 
#Using heatmap to visualize the correlation matrix
corr = sn.heatmap(corr_matrix, annot=True)

In [ ]:
corr.get_figure().savefig(path + "\\Results\\" + "corr_matrix_vlastni.pdf")